 
# Установка библиотек

In [1]:
!pip install ultralytics
!pip install clearml
!pip install supervision

DEPRECATION: pyodbc 4.0.0-unsupported has a non-standard version number. pip 24.1 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of pyodbc or contact the author to suggest that they release a version with a conforming version number. Discussion can be found at https://github.com/pypa/pip/issues/12063


DEPRECATION: pyodbc 4.0.0-unsupported has a non-standard version number. pip 24.1 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of pyodbc or contact the author to suggest that they release a version with a conforming version number. Discussion can be found at https://github.com/pypa/pip/issues/12063
DEPRECATION: pyodbc 4.0.0-unsupported has a non-standard version number. pip 24.1 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of pyodbc or contact the author to suggest that they release a version with a conforming version number. Discussion can be found at https://github.com/pypa/pip/issues/12063


In [1]:
from ultralytics import YOLO
import cv2
import numpy as np
import supervision as sv
import pandas as pd
from joblib import load

In [18]:
model = YOLO("yolov8s-pose.pt")

100%|██████████████████████████████████████| 22.4M/22.4M [00:01<00:00, 14.6MB/s]


 
# Общие геометрические функции, которые потребуются для написания эвристикdd

![](https://github.com/ybelonogov/ml_task/tree/master/data/keypoints_yolo.png)

# Вывод основной + debug информации на экран

In [3]:
def print_count(image, pull_ups, push_ups, squats, person):
    org = [20, 100]  # Например, (x, y) = (50, 50)
    font = cv2.FONT_HERSHEY_SIMPLEX
    font_scale = 4  # Масштаб шрифта
    font_color = (0, 200, 0)  # Цвет шрифта в формате BGR   
    thickness = 6  # Толщина шрифта
    org[1] = org[1] + (person - 1) * 400
    # text = "person: {0} ".format(person)
    # cv2.putText(image, text, org, font, font_scale, font_color, thickness)
    # org[1] += 100
    text = "pull ups: {0} ".format(pull_ups)
    cv2.putText(image, text, org, font, font_scale, font_color, thickness)
    org[1] += 100
    text = "push ups: {0} ".format(push_ups)
    cv2.putText(image, text, org, font, font_scale, font_color, thickness)
    org[1] += 100
    text = "squats: {0}".format(squats)
    cv2.putText(image, text, org, font, font_scale, font_color, thickness)


def print_debag_info(image, movment_down,position):
    org = [20, 100]  
    font = cv2.FONT_HERSHEY_SIMPLEX
    font_scale = 2  
    font_color = (0, 200, 0)    
    thickness = 2  
    org[1] = org[1] + 400
    text = "movment_down: {0} ".format(movment_down)
    cv2.putText(image, text, org, font, font_scale, font_color, thickness)
    org[1] += 50
    text = "movment_down predict: {0} ".format(position==1)
    cv2.putText(image, text, org, font, font_scale, font_color, thickness)


## Функция для локализации ключевых точек

In [4]:
def to_predict_format (result):
    x1 = result.boxes.xyxy[0][0].item()
    y1 = result.boxes.xyxy[0][1].item()
    x2 = result.boxes.xyxy[0][2].item()
    y2 = result.boxes.xyxy[0][3].item()
    w = x2 - x1  # ширина bounding box
    h = y2 - y1  # высота bounding box
    ans = {}
    for i, (x, y) in enumerate(result.keypoints.xy[0]):
        if x.item()!=0:
            ans[f"{i}_x"] = (x.item()-x1)/w
            ans[f"{i}_y"] = (y.item()-y1)/h
        else:
            ans[f"{i}_x"] = x.item()
            ans[f"{i}_y"] = y.item()
    return pd.DataFrame(ans, index=[0])
    

# Загрузка и обработка видео  

In [23]:
video = cv2.VideoCapture("data/video/push_ups2.mp4")
video_info = sv.VideoInfo.from_video_path("data/video/push_ups2.mp4")
fourcc = cv2.VideoWriter_fourcc(*"XVID")
output_video = cv2.VideoWriter("ans/push_ups2v2.mp4", fourcc, video_info.fps,
                               (video_info.width, video_info.height))
push_up_checker = load('push_up_checker_cat_boost.joblib')
# result = model(source="data/push_ups1.mp4", show=False, conf=0.3, save=True)
push_up_count = 0
pull_up_count = 0
squats_count = 0
movment_down = True
prev_angle = 0
cadr_num = 0

while True:
    ret, frame = video.read()
    if not ret:
        break
    if cadr_num % 10 == 0:
        results = model.predict(frame)
        person = 1
        result = results[0]
        keypoints = to_predict_format(result)
        position = push_up_checker.predict(keypoints)
        if position[0][0]==0 and movment_down:
            push_up_count +=1
            movment_down ==False
        elif position[0][0]==1 and not movment_down:
            movment_down ==True
    cadr_num += 1
    print_debag_info(frame, movment_down,position[0][0])
    print_count(frame, pull_up_count, push_up_count, squats_count, person)
    output_video.write(frame)

video.release()
output_video.release()
# cv2.destroyAllWindows()

OpenCV: FFMPEG: tag 0x44495658/'XVID' is not supported with codec id 12 and format 'mp4 / MP4 (MPEG-4 Part 14)'
OpenCV: FFMPEG: fallback to use tag 0x7634706d/'mp4v'

0: 640x384 1 person, 68.0ms
Speed: 6.5ms preprocess, 68.0ms inference, 1.4ms postprocess per image at shape (1, 3, 640, 384)

0: 640x384 1 person, 68.0ms
Speed: 2.5ms preprocess, 68.0ms inference, 1.5ms postprocess per image at shape (1, 3, 640, 384)

0: 640x384 1 person, 68.1ms
Speed: 2.7ms preprocess, 68.1ms inference, 1.4ms postprocess per image at shape (1, 3, 640, 384)

0: 640x384 1 person, 68.2ms
Speed: 2.9ms preprocess, 68.2ms inference, 1.5ms postprocess per image at shape (1, 3, 640, 384)

0: 640x384 1 person, 68.1ms
Speed: 2.9ms preprocess, 68.1ms inference, 1.7ms postprocess per image at shape (1, 3, 640, 384)

0: 640x384 1 person, 67.9ms
Speed: 3.0ms preprocess, 67.9ms inference, 1.5ms postprocess per image at shape (1, 3, 640, 384)

0: 640x384 1 person, 67.9ms
Speed: 3.0ms preprocess, 67.9ms inference, 1.5ms 

error: OpenCV(4.9.0) /io/opencv/modules/highgui/src/window.cpp:1266: error: (-2:Unspecified error) The function is not implemented. Rebuild the library with Windows, GTK+ 2.x or Cocoa support. If you are on Ubuntu or Debian, install libgtk2.0-dev and pkg-config, then re-run cmake or configure script in function 'cvDestroyAllWindows'
